# Détection de Pneumonie à l'aide de SVM


Ce notebook utilise un classifieur SVM pour détecter la pneumonie à partir d'images de radiographies pulmonaires. 
Les images sont pré-traitées pour isoler les poumons et sont ensuite utilisées pour entraîner et tester le modèle.



## Importation des bibliothèques et définition des chemins
Dans cette section, nous importons les bibliothèques nécessaires et définissons les chemins vers les ensembles de données.


In [28]:

import os
import time
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split
import cv2

# Chemins vers les ensembles de données
path_train_malades = 'C:/Users/Mentu/Documents/IA/Zoidberg/train/PNEUMONIA'
path_train_non_malades = 'C:/Users/Mentu/Documents/IA/Zoidberg/train/NORMAL'
path_test_malades = 'C:/Users/Mentu/Documents/IA/Zoidberg/test/PNEUMONIA'
path_test_non_malades = 'C:/Users/Mentu/Documents/IA/Zoidberg/test/NORMAL'
    


## Fonction de recadrage des images
Cette fonction recadre la zone des poumons dans une image pour se concentrer uniquement sur la région d'intérêt.


In [29]:

# Fonction pour recadrer la zone des poumons d'une image
def crop_lungs_from_image(img):
    """Recadre uniquement la zone des poumons d'une image.
    :param img: image à recadrer.
    :returns: image recadrée."""
    img_cv = np.array(img)
    _, thresh = cv2.threshold(img_cv, 15, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        print("No contours found")
        return img  # Return the original image if no contours are found
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    cropped_img_cv = img_cv[y:y + h, x:x + w]
    cropped_img = Image.fromarray(cropped_img_cv)
    return cropped_img
    


## Définition des fonctions utilitaires
Nous définissons ici deux fonctions : l'une pour mesurer le temps d'exécution et l'autre pour charger les images à partir de deux dossiers différents.


In [30]:

# Fonction pour mesurer le temps d'exécution des fonctions
def timing_function(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} a pris {end - start:.2f} secondes")
        return result
    return wrapper 

# Fonction pour charger des images à partir de deux dossiers
@timing_function
def load_images_from_two_folders(folder1, folder2, image_size=(90, 90)):
    images, labels = []

    # Charger les images du premier dossier
    for filename in os.listdir(folder1):
        if filename.endswith('.jpeg'): # Vérifiez l'extension du fichier si nécessaire
            img = Image.open(os.path.join(folder1, filename)).convert('L')  # Convertir en niveaux de gris
            img = crop_lungs_from_image(img)
            img = img.resize(image_size)
            images.append(np.array(img).flatten())  # Aplatir l'image
            labels.append(1)  # Étiquette 1 pour le premier dossier

    # Charger les images du deuxième dossier
    for filename in os.listdir(folder2):
        if filename.endswith('.jpeg'):  # Vérifiez l'extension du fichier si nécessaire
            img = Image.open(os.path.join(folder2, filename)).convert('L')  # Convertir en niveaux de gris
            img = crop_lungs_from_image(img)
            img = img.resize(image_size)
            images.append(np.array(img).flatten())  # Aplatir l'image
            labels.append(0)  # Étiquette 0 pour le deuxième dossier

    return np.array(images), np.array(labels)
    


## Chargement des images et des étiquettes puis Création et entraînement du modèle SVM
Nous créons un pipeline avec la mise à l'échelle des données et SVM, puis entraînons le modèle avec les données d'entraînement.


In [33]:
# Charger les images et les étiquettes d'entraînement
train_images, train_labels = load_images_from_two_folders(path_train_malades, path_train_non_malades)

# Charger les images et les étiquettes de test
test_images, test_labels = load_images_from_two_folders(path_test_malades, path_test_non_malades)

# Créer un pipeline avec la mise à l'échelle des données et SVM
pipeline = make_pipeline(StandardScaler(), SVC(probability=True, class_weight='balanced', kernel='rbf'))

# Entraîner le modèle
pipeline.fit(train_images, train_labels)
    

ValueError: not enough values to unpack (expected 2, got 0)


## Prédiction et évaluation du modèle
Nous effectuons des prédictions sur les données de test et affichons le rapport de classification.


In [ ]:

# Faire des prédictions
predicted_labels = pipeline.predict(test_images)

# Afficher le rapport de classification
print(classification_report(test_labels, predicted_labels))
    


## Matrice de confusion
Nous traçons la matrice de confusion pour visualiser les performances du modèle.


In [ ]:

# Tracer la matrice de confusion
cm = confusion_matrix(test_labels, predicted_labels)
fig, ax = plt.subplots()
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
ax.set(xticks=np.arange(cm.shape[1]),
       yticks=np.arange(cm.shape[0]),
       xticklabels=['Normal', 'Pneumonie'],
       yticklabels=['Normal', 'Pneumonie'],
       title='Matrice de Confusion',
       ylabel='Étiquette réelle',
       xlabel='Étiquette prédite')
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], 'd'),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")
plt.show()
    


## Courbe ROC
Nous calculons et traçons la courbe ROC pour évaluer les performances du modèle en termes de taux de faux positifs et de vrais positifs.


In [ ]:

# Calculer la courbe ROC et l'aire sous la courbe pour l'ensemble de test
probabilities = pipeline.predict_proba(test_images)[:, 1]
fpr, tpr, _ = roc_curve(test_labels, probabilities)
roc_auc = auc(fpr, tpr)

# Tracer la courbe ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Courbe ROC (aire = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('Caractéristique de Fonctionnement du Récepteur')
plt.legend(loc='lower right')
plt.show()
    